In [ ]:
# !pip install langdetect

In [ ]:
import pandas as pd
from langdetect import detect
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv

REVIEWS DATA CLEANING

In [ ]:
df_reviews = pd.read_csv('reviews.csv')

print(df_reviews.info())

In [ ]:
clean_reviews = df_reviews.drop(["date", "reviewer_name"], axis=1)
print(clean_reviews.info())

In [ ]:
clean_reviews['comments'] = clean_reviews['comments'].str.lower()
print(clean_reviews.info())

In [ ]:
def det(x):
    try:
        return detect(x)
    except:
        return 'unknown'

clean_reviews['Lang'] = clean_reviews['comments'].apply(det)
clean_reviews.head()

In [ ]:
clean_reviews = clean_reviews[clean_reviews["Lang"] == "en"].drop(["Lang"], axis=1)

In [ ]:
print(clean_reviews.info())

In [ ]:
clean_reviews.to_csv('clean_review.csv',index = False)

In [ ]:
clean_reviews = pd.read_csv('clean_review.csv')
# clean_reviews = clean_reviews[['listing_id', 'comments']]
clean_reviews.info()

In [ ]:
# !pip install vaderSentiment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
import matplotlib.patches as mpatches
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv

In [ ]:
c = clean_reviews.loc[:,'comments']
print(c.head())

In [ ]:

sid_obj = SentimentIntensityAnalyzer()
count = 0
sent = []
for a in c:
  n = sid_obj.polarity_scores(a)
  count = count+1
  # print(count)
  sent.append(n['compound'])


In [ ]:
clean_reviews['Sentiment']= sent
print(clean_reviews.info())

In [ ]:
clean_reviews.to_csv('clean_review.csv',index = False)
clean_reviews = pd.read_csv('clean_review.csv')
print(clean_reviews.info())

In [ ]:
# pd.set_option('max_rows', 99999)
# pd.set_option('max_colwidth',9999999)
# pd.describe_option('max_colwidth')

In [ ]:
clean_reviews = clean_reviews.sort_values('listing_id')
print(clean_reviews.info())

In [ ]:
l = clean_reviews.loc[:,'listing_id']
s = clean_reviews.loc[:,'Sentiment']

l.size

In [ ]:
new_sen = []
new_id = [] 
idx = 0
i = 0

while(i<l.size):
    cnt = 0
    sum = 0
    id = l[i]
    new_id.append(id)

    # while( i<l.size):
        # print("HERE")
    while( i<l.size and l[i]==id ):
        cnt = cnt + 1
        sum = sum + s[i]
        i = i + 1
        print(str(id) + " Count = "+ str(cnt))

    # print("HERE: "+ str(l[i])+ " "+str(id))
    
    new_sen.append(round((sum/cnt),5))
    print("ID:" + str(new_id[idx]) + " SEN:" + str(new_sen[idx]) + " SUM:" + str(sum))
    print("------------------")
    idx = idx + 1


In [ ]:
l = clean_reviews.loc[:,'listing_id']
c = clean_reviews.loc[:,'comments']

new_comm = []
temp_id = [] 
idx = 0
i = 0

while(i<l.size):
    cnt = 0
    all_comm = " "
    id = l[i]
    temp_id.append(id)

    # while( i<l.size):
        # print("HERE")
    while( i<l.size and l[i]==id ):
        cnt = cnt + 1
        all_comm = all_comm + c[i]
        i = i + 1
        print(str(id) + " Count = "+ str(cnt))

    # print("HERE: "+ str(l[i])+ " "+str(id))
    
    new_comm.append(all_comm)
    print("ID:" + str(temp_id[idx]) + " LENGTH:" + str(len(all_comm)))
    print("------------------")
    idx = idx + 1

In [ ]:
print(len(new_sen),len(new_id),len(new_comm))
# new_comm[0]

final_reviews = pd.DataFrame({'id':new_id, 'comments':new_comm, 'sentiment': new_sen})
final_reviews.shape


In [ ]:
final_reviews.to_csv('final_reviews.csv',index = False)
final_reviews = pd.read_csv('final_reviews.csv')
final_reviews.shape

### LISTING DATA CLEANING

In [ ]:
df_listing = pd.read_csv('listings.csv')


In [ ]:
df_listing=df_listing.drop(["listing_url", "scrape_id", "last_scraped", "source", "name", "picture_url", "host_id", "host_name", "host_url",
"host_thumbnail_url", "host_picture_url", "neighbourhood_group_cleansed", "bathrooms", "license", "host_location", "host_since","first_review","last_review"], axis=1)

df_listing = df_listing.drop(['neighbourhood', 'neighbourhood_cleansed','calendar_updated','calendar_last_scraped'],axis=1)

df_listing = df_listing.drop(['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'],axis=1)

df_listing = df_listing.drop(['neighborhood_overview', 'host_about', 'host_response_time', 'host_acceptance_rate','host_neighbourhood'],axis=1) 


In [ ]:
df_listing.columns.size

In [ ]:
X=df_listing.loc[:,['id','latitude','longitude']]
df_listing = df_listing.drop(['latitude', 'longitude'],axis=1)


X.head(10)

In [ ]:

# K_clusters = range(1,10)
# kmeans = [KMeans(n_clusters=i) for i in K_clusters]
# Y_axis = X[['latitude']]
# X_axis = X[['longitude']]
# score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]# Visualize
# plt.plot(K_clusters, score)
# plt.xlabel('Number of Clusters')
# plt.ylabel('Score')
# plt.title('Elbow Curve')
# plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 7, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['cluster_label'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
X.head(10)

In [ ]:
# X.plot.scatter(x = 'latitude', y = 'longitude', c=labels, s=50, cmap='viridis')
# plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
X = X.drop(['latitude', 'longitude'],axis=1)


df_listing = df_listing.merge(X, left_on='id', right_on='id')
# df_listing.head(5)

In [ ]:
df_listing.info()

In [ ]:
df_listing['host_response_rate'] = df_listing['host_response_rate'].str.replace('%', '')
df_listing['host_response_rate'] = df_listing['host_response_rate'].astype(float)

In [ ]:
# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.host_response_rate)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_rating)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_accuracy)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_cleanliness)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_checkin)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_communication)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_location)

# fig, ax = plt.subplots(figsize=(10,6))
# sns.distplot(df_listing.review_scores_value)

In [ ]:
# mean_value_1=round(df_listing['host_response_rate'].mean(),3)

# mean_value_2=round(df_listing['review_scores_rating'].mean(),3)

# mean_value_3=round(df_listing['review_scores_accuracy'].mean(),3)

# mean_value_4=round(df_listing['review_scores_cleanliness'].mean(),3)

# mean_value_5=round(df_listing['review_scores_checkin'].mean(),3)

# mean_value_6=round(df_listing['review_scores_communication'].mean(),3)

# mean_value_7=round(df_listing['review_scores_location'].mean(),3)

# mean_value_8=round(df_listing['review_scores_value'].mean(),3)

In [ ]:
# df_listing['host_response_rate'].fillna(value=mean_value_1, inplace=True)
# df_listing['review_scores_rating'].fillna(value=mean_value_2, inplace=True)
# df_listing['review_scores_accuracy'].fillna(value=mean_value_3, inplace=True)
# df_listing['review_scores_cleanliness'].fillna(value=mean_value_4, inplace=True)
# df_listing['review_scores_checkin'].fillna(value=mean_value_5, inplace=True)
# df_listing['review_scores_communication'].fillna(value=mean_value_6, inplace=True)
# df_listing['review_scores_location'].fillna(value=mean_value_7, inplace=True)
# df_listing['review_scores_value'].fillna(value=mean_value_8, inplace=True)

In [ ]:
# df_temp = df_listing.copy
df_temp = df_listing.loc[:,['bedrooms','beds','host_response_rate','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','reviews_per_month']]

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [ ]:
linearreg = LinearRegression()
imp = IterativeImputer(estimator=linearreg,missing_values=np.nan, max_iter=100, verbose=2, imputation_order='roman',random_state=0)
ImputedData=imp.fit_transform(df_temp)

In [ ]:
# df_temp['host_response_rate'].isna().sum()
ImputedData = pd.DataFrame(ImputedData)

In [ ]:
# !pip install miceforest
# !pip install missingpy

In [ ]:
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors=2)
ImputedData1 = knn.fit_transform(df_temp)
ImputedData1 = pd.DataFrame(ImputedData1)

In [ ]:
arr = ['bedrooms','beds','host_response_rate','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','reviews_per_month']
i = 0

for ele in arr:
    mean_value_1 = round(df_temp[ele].mean(),3)
    mean_value_2 = round(ImputedData[i].mean(),3)
    mean_value_3 = round(ImputedData1[i].mean(),3)
    i=i+1

    print(mean_value_1,mean_value_2,mean_value_3)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.distplot(ImputedData[7])

fig, ax = plt.subplots(figsize=(10,6))
sns.distplot(ImputedData1[7])

fig, ax = plt.subplots(figsize=(10,6))
sns.distplot(df_temp.review_scores_value)

# ['host_response_rate','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
# 'review_scores_communication','review_scores_location','review_scores_value']]

In [ ]:
i = 0

for ele in arr:
    df_temp[ele] = ImputedData1[i]
    i=i+1

# df_temp

In [ ]:
for ele in arr:
    df_listing[ele] = df_temp[ele]

df_listing.info()

In [ ]:
X=df_listing.loc[:,['id','bathrooms_text']]
bathrooms_text_map = {'0 shared baths':1,
                      '0 baths': 1,
                      'Shared half-bath': 2,
                      'Half-bath':3,
                      'Private half-bath':4,
                      '1 shared bath':5,
                      '1 bath': 6,
                      '1 private bath': 7,
                      '1.5 baths': 9,
                      '1.5 shared baths': 8,
                      '2 shared baths':10,
                      '2 baths':11,
                      '2.5 shared baths': 12,
                      '2.5 baths': 13,
                      '3 shared baths': 14,
                      '3 baths': 15,
                      '3.5 shared baths': 16,
                      '3.5 baths': 17,
                      '4 shared baths': 18,
                      '4 baths': 19,
                      '4.5 baths': 20,
                      '5 baths': 21,
                      '5.5 baths': 22,
                      '6 shared baths': 23,
                      '6 baths': 24,
                      '6.5 baths': 25,
                      '7 baths': 26,
                      '7.5 baths': 27,
                      '8 baths': 28,
                      '8.5 baths': 29,
                      '9.5 baths': 30}

X['bathroom_map'] = X.bathrooms_text.map(bathrooms_text_map)
X.head()

In [ ]:
df_listing = df_listing.drop(['bathrooms_text'], axis=1).merge(X, left_on='id', right_on='id')
df_listing['bathroom_map'] = df_listing['bathroom_map'].fillna(6)
df_listing = df_listing.drop(['bathrooms_text'], axis=1)
df_listing.info()

In [ ]:
df_listing.to_csv('final_listing.csv',index = False)
final_listing = pd.read_csv('final_listing.csv')
final_listing.shape

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


In [ ]:
df_listings_final = pd.read_csv('final_listing.csv')

In [ ]:
# convert text to lowercase.
import re

def convert_empty_to_nan( df, column_name ) :
    df[ column_name ] = df[ column_name ].str.lower()
    df[ column_name ] = df[ column_name ].fillna("NA") 
    df[ column_name ] = df[ column_name ].str.replace("[^a-zA-Z0-9]+", ' ', regex=True)
    # re.sub(r"[^a-zA-Z0-9]+", ' ', k)
    df[ column_name ] = df[ column_name ].str.replace('\d+', '', regex=True)
    #.replace(r'^\s*$', "No description", regex=True)

convert_empty_to_nan( df_listings_final, 'description' )

# df_listings_final['description'].head()

# df_listings_final.to_csv( '../data/listingsDesc.csv', index=False )

In [ ]:
# Removing stop words (un-necessary words) - using nltk's pre-defined stop words
STOP_WORDS = set( stopwords.words('english') )
STOP_WORDS.add("br")
# print('Stop word list:')
# print('----------------')
# print(STOP_WORDS)

def remove_stop_words(content):
   return " ".join([text for text in str(content).split() if text not in STOP_WORDS])

df_listings_final['description'] = df_listings_final['description'].apply(lambda content: remove_stop_words(content=content))

# df_listings_final['description'].head()

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

def stemming(x):
    words = word_tokenize(x)

    for i in range(len(words)):
        words[i] = ps.stem(words[i])

    return (' '.join(map(str, words)))

df_listings_final['description'] = df_listings_final.apply(lambda row : stemming(row['description']), axis = 1)
df_listings_final.description.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
# result = tfidf.fit_transform(string)

temp = df_listings_final
temp_f = temp['description']
# print(temp['description'])
x = tfidf.fit_transform(temp['description'])

df1 = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())
# print(df1)



In [ ]:
df1.shape

In [ ]:
term = []
score = []
for ele1, ele2 in zip(tfidf.get_feature_names(), tfidf.idf_):
    term.append(ele1)
    score.append(ele2)

term_scores = pd.DataFrame({'term':term, 'score':score})
term_scores.shape

In [ ]:
term_scores = term_scores.sort_values(by=['score'],ascending=False)
print(term_scores)


In [ ]:
df_listings_final = pd.read_csv('final_listing.csv')

df_listings_final = df_listings_final.drop(['description'],axis=1)

# df_listings_final = df_listings_final.drop(['room_type'],axis=1)
# df_listings_final = df_listings_final.drop(['property_type'],axis=1)
# df_listings_final = df_listings_final.drop(['host_verifications'],axis=1)
# df_listings_final = df_listings_final.drop(['amenities'],axis=1)

print(df_listings_final.shape)

final_reviews = pd.read_csv('final_reviews.csv')
final_reviews = final_reviews.drop(['comments'],axis=1)
print(final_reviews.shape)

In [ ]:
df_listings_final['price'] = df_listings_final['price'].str.replace('$', '')
df_listings_final['price'] = df_listings_final['price'].str.replace(',', '')
df_listings_final['price'] = df_listings_final['price'].astype(float)

In [ ]:
df_listings_final['host_is_superhost'] = df_listings_final['host_is_superhost'].map({'t': 1, 'f': 0})
df_listings_final['host_has_profile_pic'] = df_listings_final['host_has_profile_pic'].map({'t': 1, 'f': 0})
df_listings_final['host_identity_verified'] = df_listings_final['host_identity_verified'].map({'t': 1, 'f': 0})
df_listings_final['has_availability'] = df_listings_final['has_availability'].map({'t': 1, 'f': 0})
df_listings_final['instant_bookable'] = df_listings_final['instant_bookable'].map({'t': 1, 'f': 0})

In [ ]:
final_df = pd.merge(df_listings_final,final_reviews,left_on='id',right_on='id')

# final_df = final_df.drop(['id'],axis=1)

print(final_df.shape)

In [ ]:
# from sklearn import preprocessing

# x = final_df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)

# print(df)

In [ ]:
final_df.info()

In [ ]:
def replace_Number(content):
   return len(content.split(','))

def punctuation_processing(content):
    return str(content).translate(str.maketrans('', '', '!"#$%&\'()*+-./:;<=>?@[\]^_`{|}~\\'))


X=df_listings_final.loc[:,['id','host_verifications']]
X['host_verifications'] = X['host_verifications'].apply(lambda content: punctuation_processing(content=content))
X['host_verifications_count'] = X['host_verifications'].apply(lambda content: replace_Number(content=content))
X = X.drop(['host_verifications'], axis=1)

final_df = final_df.merge(X, left_on='id', right_on='id')
final_df = final_df.drop(['host_verifications'], axis=1)

final_df.info()

In [ ]:
X=df_listings_final.loc[:,['id','amenities']]
X['amenities'] = X['amenities'].apply(lambda content: punctuation_processing(content=content))
X['amenities_count'] = X['amenities'].apply(lambda content: replace_Number(content=content))
X = X.drop(['amenities'], axis=1)

final_df = final_df.merge(X, left_on='id', right_on='id')
final_df = final_df.drop(['amenities'], axis=1)

final_df.info()

In [ ]:
final_df['room_type'] = final_df['room_type'].map({'Shared room': 1, 'Hotel room': 2, 'Private room': 3, 'Entire home/apt': 4})
final_df = final_df.drop(['property_type'],axis=1)


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=f_regression, k=10)

X=final_df
X=X.drop('review_scores_location',axis=1)
y = pd.DataFrame(final_df['review_scores_location'])

# X = final_df.iloc[:,0:19]  #independent columns
# y = final_df.iloc[:,20]
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(50,'Score'))  #print 10 best features

In [ ]:
# import pandas as pd
# import numpy as np
# data = final_df
# X = data.iloc[:,0:19]  #independent columns
# y = data.iloc[:,20]    #target column i.e price range
# from sklearn.ensemble import ExtraTreesClassifier
# import matplotlib.pyplot as plt
# model = ExtraTreesClassifier()
# model.fit(X,y)
# print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
# #plot graph of feature importances for better visualization
# feat_importances = pd.Series(model.feature_importances_, index=X.columns)
# feat_importances.nlargest(10).plot(kind='barh')
# plt.show()

'host_response_time', 'host_acceptance_rate' fill and try
